##João Augusto Silva Bottrel - RID0468 (Dinamica DNC DEX Day_07)

Objetivo: Criar um modelo de recomendação baseado em um histórico de vendas, produtos e reviews de uma plataforma de e-commerce

#0. Importando Bibliotecas

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

#1. Importando o Dataset

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
path1='/content/gdrive/MyDrive/Colab Notebooks/Bases/DNC_order_items_dataset.csv'
df_items=pd.read_csv(path1,index_col='Unnamed: 0')

In [4]:
path2='/content/gdrive/MyDrive/Colab Notebooks/Bases/DNC_order_reviews_dataset.csv'
df_reviews=pd.read_csv(path2,index_col='Unnamed: 0')

In [5]:
path3='/content/gdrive/MyDrive/Colab Notebooks/Bases/DNC_orders_dataset.csv'
df_orders=pd.read_csv(path3,index_col='Unnamed: 0')

In [6]:
path4='/content/gdrive/MyDrive/Colab Notebooks/Bases/DNC_products_dataset.csv'
df_products=pd.read_csv(path4,index_col='Unnamed: 0')

#2. Análise inicial dos dados

##2.1 Dataset de itens

In [7]:
df_items.sample(5)

,order_id,order_item_id,product_id,price
29931,4409a8e68e232a6718d156e93a58f3ec,1,acdf83e7694c8eec22e6b19bc59f4e60,30.6
55741,7ec1a2c6522b9fcb54c68a4f8ff6f331,1,2b939dc9b176d7fa21594d588815d4a4,77.5
67114,996763a377a7e96ea28f02058d1e1883,2,90aaf7f993470e411207adabf5b998b6,199.0
57542,82eba8671de9a5c76c7f3edfcd993955,1,1cee481bc2bde9c49bba35c615d52e78,34.9
8923,1469262ec8414d0e3f821bda79cf14cc,1,59fb1c8d9f3640a622cf774c3867e170,38.9


O dataset de itens possui como variáveis:        
* order_id = identificador do pedido de compra,
* order_tem_id = número de produtos em uma mesma compra,
* product_id = identificador do produto

In [8]:
df_items['order_id'].value_counts()

8272b63d03f5f79c56e9e4120aec44ef    21
ab14fdcfbe524636d65ee38360e22ce8    20
1b15974a0141d54e36626dca3fdc731a    20
9ef13efd6949e4573a18964dd1bbe7f5    15
428a2f660dc84138d969ccd69a0ab6d5    15
                                    ..
f9207b5726780645ebba8401ccc336c6     1
ff07ff058c25c993a564127557043c5a     1
bc2e4d7ad808e01f4e639fc4c447966a     1
d15a4add9a1af2006ad10d70d9a4b634     1
9827d07ad9392a9fff10d7b9f6c05f7e     1
Name: order_id, Length: 98666, dtype: int64

O resultado acima indica que tem alguns pedidos de compra que possui diversos produtos

In [9]:
df_items.shape #Verificar o tamanho do dataset

(112650, 4)

##2.2 Dataset de reviews

In [10]:
df_reviews.sample(5)

,review_id,order_id,review_score
71041,3fbb9f98fa47e6456c2e228882107944,0c31ed220260a476cfe59bbb6608b7ad,4
86656,e8b089ecc2f8aa46c9893626bae20cf8,0067c6030d65af0ff98790158ddb0020,5
42037,676890677144bf02e6674985b3b620cd,471203d247e323c6eeb7ddbea92e69f4,3
45176,9e25d6e3025e9b9a0fc7f03588d33e2b,869997fbe01f39d184956b5c6bccfdbe,1
82937,2bc3e0fbe51f83f0a4ff9ae1852b01f9,34d92634e60b8234ac42750fd4b76fec,4


O dataset de reviews possui como variáveis:        
* order_id = identificador do pedido de compra,
* review_id = identificador da revisão do pedido de compra,
* review_score = nota ( 1 a 5) que o consumidor atribuiu ao pedido de compra

In [11]:
df_reviews.shape #Verificar o tamanho do dataset

(99224, 3)

##2.3 Dataset de ordens



In [12]:
df_orders.sample(5)

,order_id,customer_id,order_status
37141,e0ef63a06438a4f6dfa047fc41217f8e,3faca6b775e45e3b971ab1263ef1c2ec,delivered
13458,052b35671e2bdd45aa889c5fa09e2809,29509a171c5d01602fbd7afb4ecbfa06,delivered
8179,72fb560d115ecf3b15de9b853c02e505,d90af5c00814430fc3e212e8163bf2b8,delivered
82798,6e1fdac1871d58f1e81976be36a2634a,b1025fe116e2906edeb529c659bac499,delivered
77248,390056468c27192af8354d708051054e,c450d2558dd77bc0cd8b290f5af4f5fa,delivered


O dataset de orders possui como variáveis:        
* order_id = identificador do pedido de compra,
* customer_id = identificador do usuário que efetuou a compra,
* order_status = status da entrega do produto

In [13]:
df_orders.shape #Verificar o tamanho do dataset

(99441, 3)

##2.4 Dataset de produtos

In [14]:
df_products.sample(5)

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
19284,35ffbac108d291eae9a8df90f92f6fad,eletrodomesticos,54.0,186.0,950.0,62.0,17.0,25.0
27156,c7a94c6dde9d81dc864941084dbdfe45,esporte_lazer,58.0,246.0,1100.0,60.0,15.0,15.0
10049,92b82f3b6d4670bba0ae207094f1d226,moveis_decoracao,45.0,96.0,10100.0,60.0,10.0,40.0
17993,f6a84928571f68635beb0f5c9333c6f3,esporte_lazer,60.0,1247.0,30000.0,86.0,66.0,36.0
5323,ec6bda3514b2616da08c5139893b14d2,moveis_decoracao,23.0,468.0,800.0,16.0,5.0,16.0


O dataset de orders possui como variáveis:        
* product_id = identificador do produto,
* product_category_name = Categoria do produto,
* product_name_length = Tamanho do nome do produto
* product_description_length = comprimento do produto
* products_weight_g = peso do produto em g
* products_lenght_cm = comprimento do produto em cm
* products_height_cm = Altura do produto em cm
* product_widht_cm = largura do produto em cm

In [15]:
df_products.shape #Verificar o tamanho do dataset

(32951, 8)

In [16]:
df_products['product_category_name'].value_counts() #Avaliando quantidade de produtos por categoria

cama_mesa_banho                  3029
esporte_lazer                    2867
moveis_decoracao                 2657
beleza_saude                     2444
utilidades_domesticas            2335
                                 ... 
casa_conforto_2                     5
fashion_roupa_infanto_juvenil       5
pc_gamer                            3
seguros_e_servicos                  2
cds_dvds_musicais                   1
Name: product_category_name, Length: 73, dtype: int64

#3 Tratamento e limpeza dos dados

Aqui vamos unir os datasets de items e reviews utilizando o order_id como chave em seguida repetir o procedimento para o dataset de orders

In [17]:
df4 = pd.merge(df_items,df_reviews,how='inner',on='order_id')

In [18]:
df4.isnull().sum()

order_id         0
order_item_id    0
product_id       0
price            0
review_id        0
review_score     0
dtype: int64

In [19]:
df3 = pd.merge(df4,df_orders,how='inner',on='order_id')

In [20]:
df3.head()

,order_id,order_item_id,product_id,price,review_id,review_score,customer_id,order_status
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,97ca439bc427b48bc1cd7177abe71365,5,3ce436f183e68e07877b285a838db11a,delivered
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,7b07bacd811c4117b742569b04ce3580,4,f6dd3ec061db4e3987629fe6b26e5cce,delivered
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,0c5b33dea94867d1ac402749e5438e8b,5,6489ae5e4333f3693df5ad4372dab6d3,delivered
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,f4028d019cb58564807486a6aaf33817,4,d4eb9395c8c0431ee92fce09860c5a06,delivered
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,940144190dcba6351888cafa43f3a3a5,5,58dbd0b2d70206bf40e62cd34e84d795,delivered


A seguir iremos unir com o dataset de produtos utilizando o ID do produto como chave

In [21]:
df2 = pd.merge(df3,df_products,how='inner',on='product_id')

In [22]:
df2.sample(5)

,order_id,order_item_id,product_id,price,review_id,review_score,customer_id,order_status,product_category_name,product_name_lenght,product_description_lenght,product_weight_g,product_length_cm,product_height_cm,product_width_cm
11706,c33af7b181d4da454737cbdb08450735,1,93c902b021a9e594f658ab1b0351602a,103.9,0adde70f9eee99e90bed92eb26c53ab9,5,f1b50ffada8a14977978bac3330283e0,delivered,eletronicos,60.0,390.0,1250.0,26.0,10.0,18.0
60606,acc957c82b00d1fb5b75a1f27a6d5a5c,1,b1f32fc9713cf8c0813fff7c9c8262b2,45.6,0d87f9a1ad53953dc4dbbfefdbe578a7,5,f0475449a5e7db1fdc4148ea907fda6f,delivered,informatica_acessorios,46.0,644.0,225.0,30.0,9.0,23.0
88049,58527ee4726911bee84a0f42cdd797c1,1,2b284ca976943311c427da3bba8f47ff,129.9,10bf378a2b493cc2734e2ceae80e689d,3,b7d68eb92ede54186f0385024c0481bf,delivered,esporte_lazer,47.0,524.0,750.0,23.0,28.0,38.0
106283,c2411b9c7822d227e3e48018405a0594,1,1a0f5bca5c36a68073930270bb47e314,1089.0,bddae947012d62dd70facc32a4742049,4,464ad1fa19f2a649e39193e5c98b8300,delivered,esporte_lazer,54.0,1403.0,15600.0,55.0,26.0,29.0
69998,284d9d5dfbec3d66bf57c38628892577,1,257726ef77c81150339893567c4de6cd,49.0,27452f1e860409227dedd1c07680f86a,5,7d831cbb1fb8d8d06cc4391b76869b64,delivered,moveis_decoracao,60.0,1969.0,400.0,69.0,11.0,11.0


Vamos utilizar como métrica as reviews dos usuários para criar um método de recomendação utilizando um filtro colaborativo com um fallback dos produtos mais bem avaliados. Por isso, iremos remover as colunas que tem as caractéristicas como tamanho do produto ou quantidade de letras utilizadas na descrição, pois não agregam para a recomendação

In [23]:
df1 = df2.drop(columns=['order_id','order_item_id','price','review_id','order_status','product_name_lenght','product_description_lenght','product_weight_g','product_length_cm','product_height_cm','product_width_cm']).reindex(columns=['customer_id','product_id','product_category_name','review_score'])

In [24]:
df1.sample(10)

,customer_id,product_id,product_category_name,review_score
27427,1732351234779d12dcc355581d02d7b9,a9fb80187bf64d19dd2804f25e83c5e6,brinquedos,5
102233,fbbbf9c39662ec023c52d0169700b4d6,efa9bf5559d026b9872259f29b7304dc,utilidades_domesticas,3
62132,708761cac9260fe06a7ac1e921379700,fc5bf9e50dbfb45e7634bd8c045411c5,cool_stuff,1
86440,19164000c4f2f70911bb28eb4b20325a,fb98e86cfbfbff699907f0a84784d474,cama_mesa_banho,5
99735,b24e76ff960c3a5cf88ce8852d6e9fe8,d4a3a15f99c94c6741d16d3375b928e6,relogios_presentes,4
2016,af88f4b4935d45b8c2173518c445c249,fe077ec80df6b4ee60bb4498d5ab1962,informatica_acessorios,5
69032,d98edf3a73e942ed1ece36e5228783f5,8af72bbf3875865c7fc23509cf49682e,brinquedos,4
82854,2967800a1e6a657df2f13ef37b65c9ba,f713c90e84b1bf2e157637d63fe09a68,cool_stuff,4
7154,7456113cce8651aa1338725882c6a353,78efe838c04bbc568be034082200ac20,moveis_decoracao,5
65682,135ed82d00d601dd15e861f2bf1deb93,5f0a49e6e539d4e186eeddfd69db9863,agro_industria_e_comercio,5


In [25]:
df1.shape

(112372, 4)

Abaixo é realizado um tratamento de nulos de produtos sem categoria

In [26]:
df1.isnull().sum()

customer_id                 0
product_id                  0
product_category_name    1598
review_score                0
dtype: int64

In [27]:
df1 = df1.dropna()

In [28]:
df1.sample(5)

,customer_id,product_id,product_category_name,review_score
20152,9bca1f1ee89cfc3207037cc520ac5978,4413a608a42bd21bbf9d5abbd58c249a,cool_stuff,3
49395,0be39985ec47a13a76f780957f516a6c,151133751552ad5bdd6645dfdcb0e728,cama_mesa_banho,1
56670,1596f3450e07b5743263cf514cb9c829,78018501054dd80dc909eee2f22ddbd4,cool_stuff,4
91418,642f858f2d65c052cfa9058e21181a88,be958064244f19bf11ee270b2feca7a2,moveis_decoracao,4
70159,f3dc55ecb71ebe4066a6efc57c6593b3,f9b4f8ef12259efd919fbaf21f8f66a4,cama_mesa_banho,5


Supondo que o usuário entre na categoria "cama_mesa_banho" da plataforma de e-commerce, vamos criar uma matriz de recomendação que retorne itens somente dessa categoria, removendo as demais

In [29]:
df = df1[df1['product_category_name']=='cama_mesa_banho'].drop(columns='product_category_name')

In [30]:
df.sample(5)

,customer_id,product_id,review_score
81747,3baded4af7acec322d1c4eb8b405a0b4,a3fd6c4ecde9035c3e8dd4e0fd8b8048,5
85088,deba7800cadd368aa9a7fb477f78864e,872c150bc3167f3a4fd9214f82931358,5
89418,158195ad490cea5aee2cd7e3e85fa3b4,4ee931ba1980085a969d498d63e69f72,5
110070,91b3ab92e7279abc4955341318de42f2,387f1559343d513c4383bdb99672e431,1
74213,9582b93f8a5b04336d13454b876e72ba,b570f2a8228eeaa87e224f25264576b5,1


In [31]:
df.shape

(11137, 3)

In [32]:
df['product_id'].value_counts()

99a4788cb24856965c36a24e339b6058    482
f1c7f353075ce59d8a6f3cf58f419c9c    152
06edb72f1e0c64b14c5b79353f7abea3    146
ec2d43cc59763ec91694573b31f1c29a    130
84f456958365164420cfc80fbe4c7fab    107
                                   ... 
baf59a3a8f15ea34f04394f44bad573a      1
a54f6200fb09c904053b05461c2e91f8      1
334f9a7bc70ecefea3a1fc7dd1c7ec76      1
9f6b6b828b980352cbd59ff7a03a7277      1
48ef156012e010507e2e96a9ef9e79f0      1
Name: product_id, Length: 3009, dtype: int64

In [33]:
df.dtypes

customer_id     object
product_id      object
review_score     int64
dtype: object

Esse é o dataset que servirá de base para o modelo de recomendação

#4. Modelo de recomendação

##4.1 Matriz de recomendação

Implementando um grupo de funções para realizar um agrupamento de produtos por contagem, um agrupamento dos produtos pela média de reviews e a seleção dos n melhores produtos

In [34]:
def start_pipeline(dataf):
    return dataf.copy()

def get_product_counts(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'customer_id': pd.Series.nunique}) \
    .rename(columns={'customer_id': 'customer_id_count'}) \
    .set_index('product_id') \
    .sort_values('customer_id_count', ascending = False)

def get_avg_ratings(dataf):
  return dataf.groupby(by='product_id', as_index=False) \
    .agg({'review_score': np.mean}) \
    .rename(columns={'review_score': 'review_score_avg'}) \
    .set_index('product_id') \
    .sort_values('review_score_avg', ascending = False)

def get_n_top_values(dataf, n):
  return dataf.head(n) \
    .index \
    .tolist()

top_product_ratings = df.pipe(start_pipeline).pipe(get_avg_ratings).pipe(get_n_top_values, n=1000) #o n irá determinar o tamanho da matriz de relacionamento

top_product_sellers = df.pipe(start_pipeline).pipe(get_product_counts).pipe(get_n_top_values, n=10)

In [35]:
top_product_ratings[:10], top_product_sellers

(['7e8a5e8789ede63ab600b65235e35fa6',
  '7bb73fff8e232153ed20fef83e34e84c',
  '7b7867b1566350578959e705c5a46b2f',
  '7b564d6b4c926ddf0a111006209e1088',
  '7b1f55eb9b85b0aa6b1d4c16108eac61',
  '7b07f25dab76a821beca96d40ee0553e',
  '7ad1208884270048a2176ff437f49a9b',
  '7aac56e58e89831fecb41998f93049ea',
  '7a6750f9cb558b9fc79dbfa0d121a589',
  '79e193444d3b3ba2e2f28d0e0f9bd97a'],
 ['99a4788cb24856965c36a24e339b6058',
  'f1c7f353075ce59d8a6f3cf58f419c9c',
  '06edb72f1e0c64b14c5b79353f7abea3',
  'ec2d43cc59763ec91694573b31f1c29a',
  '84f456958365164420cfc80fbe4c7fab',
  '777d2e438a1b645f3aec9bd57e92672c',
  '363218ba55c610b750224f90bdd34be1',
  '64fb265487de2238627ce43fe8a67efc',
  '2a2d22ae30e026f1893083c8405ca522',
  'bd6e6fce9ada76ea2db0f1912e8e478f'])

In [36]:
df.pipe(start_pipeline).pipe(get_avg_ratings)

,review_score_avg
product_id,
7e8a5e8789ede63ab600b65235e35fa6,5.0
7bb73fff8e232153ed20fef83e34e84c,5.0
7b7867b1566350578959e705c5a46b2f,5.0
7b564d6b4c926ddf0a111006209e1088,5.0
7b1f55eb9b85b0aa6b1d4c16108eac61,5.0
...,...
9f4a55260fae788c26fcbb2b934ffa52,1.0
9f683084a0b0211109fef6e538cb4767,1.0
86304b9e4529f2686bc91f37c6392801,1.0


Criando a matriz relacionando produtos com usuários

In [37]:
matriz_de_relacionamento = df[df["product_id"].isin(top_product_ratings)].pivot_table(index='product_id', columns='customer_id', values='review_score').fillna(0)

matriz_de_relacionamento.shape

(1000, 1203)

In [38]:
matriz_de_relacionamento.sample(5)

customer_id,00674fe0f6a6f190bd022dbca99795dc,00e4dd8d5ecaa00de8c6671679b58045,0125bee4b30f40607261d06f47b0f38d,0130b37ed963c4a595792582025bdcb0,0155478730861c69019d0a41d9bc98f6,01f122c4617c17afeba8f9bf49babfbc,0222b08d0816bba0acbcc31ba1d6cc1a,02d12adf94e256f7ff0056d447557027,02e04e4adf1a4ec2e262f8f942b4b315,02e0c77c1d6bd78e21d61a7c7b818af9,02f0bc4b5ee293031637b7e8d75a5484,0303e5527d4342ac7939d2bfc0f9f9e2,0308c8188ffa174ae335cf15234bef98,0311e8645872f2e940fcdbd0373e8ef3,0320b57550ef2aea72c685cc12c09ecd,032dc4140d9a51a0febc693a96a44489,032f5eb3f82038a0490fd6870bf131dd,03a94d235a422b784953fdbad346162e,03f2849096ae58cf4ef032d9a4bd72a0,045113dd9def11da63eed78d7bd66571,0499e8d702425ef8b10da2172f9fec96,04a80904d68808014ce304276a945dc2,04ebc2712225141ac34d1ba5ebee0629,0502d707915775b984e0976f86be4e70,0554357bede8e7adab8e4d3a88652483,05c9c8bee01dca6e97b35284f55a6a34,05e53f1437a94e4fa5dbef5eb5c0fcff,05ebc27279c0c9082b56bbd6a1441a4b,05fe48df6ea41f403217b5cb88126ce0,060d1aa4818f0382d6a4cd5549e63dc4,06a70917afd2dcd59396e1eac836c646,06c760a59b5508e58bb333d3bac7afc9,06d99479baac3413561b21bd7ac27767,06e30ee84a0678d353928f7306faac30,0712358f39f7ee528dd7930c5e43b4db,07396c453f4681f7be214112539b29b8,073d9752205a02fe743263d3d781cce1,0787afe87fbc7be9114a66325d1a5481,078a3d6f31fa48e82be1142201dc5fc3,07bb78d58680e12babcfcb21476f9e14,...,f5d8a545323f03c74139eb656ad7c355,f5fb123023cbd4acdc614393621b6f86,f60343ab919f4c92c6684d9fff0ed1ee,f632ad5299fd1a88f9aed3c617d761d0,f66f2668857fa2ede0a1d2147fe10bde,f6af2b95b6be76fe88673d0a62a7db2f,f6c2d6d91ed17d749ad0a5ffd23f710e,f6e6c22f0f45f1e57d6eb674bc6fdb3b,f6f76301950a9f0c4099483fa35b0292,f718a2c31336e1bd948bf62c2424ff62,f72224709e8bef3430441583b08a2d64,f777873517b1bf70c51a2950e513acf6,f7ca9d963c6dcc2b5c4b93006b6abe90,f80fc8c638c8616af8789ae5e751ff13,f8e2c69fceb19e32d3bf44633ef40f5f,f9055270b5abc752526d3bb0a5a68cc3,f97b46627a4e56ccc5f4026cdab2b60f,f99b5ad76ac1085ae3628a3dc0eff0af,f9dfd7942a2d7f1588b7638a9bdc8ff1,fa3e77981c620b52d92ee2d390543f63,fa8a4bda569c794150f39cda0dfe0734,fad867ccc296df4399f09533c10bb693,fb7481b0cc9f7ef408c625a696cf3e51,fc081ae245389c9660e0fdc15c33a804,fc4476d400c4b3bf7e64456fb82626fb,fc72a2bc404989848c8afa35fab9accc,fc821401a1841453aef1515d55b09399,fd1113e9d9cf81c3b1e1704b6ae6286b,fd2ae8c714a773f85ef1b53fbebeda1f,fdea9d678bf2979af34b835a4349b904,fe082a45e6e64db9443e39ef7e3d415a,fe0d2c3b6048199a658d47a9c023d81f,fe20e5d9ad97fe0bf421a94b7a17da6f,fe3fe44146243711a4f5eabc7e65b6dc,feac61f9b2103facafb08f447664c598,febc8698a75e85a4ddf0e78926c1119c,fef445423df985e112e1b8b357f1cbee,ff73066003e6e0aa6ce3739d60a1ff24,ffa9419903b6e1ed6f0692ab2695de94,ffd294eea523f67ee8c743d3a904ee20
product_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
062907032db5e0c065bab136233a9544,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5ef15970297e8282ea1f0e3ddc528503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
57f68d17b05e574df89591b08dab6f6a,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
739e8ddbbd7a0d61e33949614a14d8cb,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,

##4.2 Similaridade de cossenos

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

cos_products = cosine_similarity(matriz_de_relacionamento)

print(cos_products)
cos_products.shape

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


(1000, 1000)

Função de recomendação

In [40]:
def recomendar(id_produto):
  print(f"Produto de interesse: {id_produto}")
  try:
    idx = matriz_de_relacionamento.index.tolist().index(id_produto)
    closest_10_products = np.argsort(-cos_products[idx])[1:11]
    print(f"Retornando 10 produtos mais próximos de: {id_produto}.")
    return matriz_de_relacionamento.index[closest_10_products].tolist()
  except ValueError as e:
    print(f"O item {id_produto} não está incluso na matriz de recomendação. Retornando os 10 mais avaliados") #retorno caso o ID não esteja na matriz de relacionamento
    return top_product_sellers

##4.3 Teste do modelo

Utilizando um ID existente

In [41]:
id = 'e76572c2ae4cede1e3b2ead512423efe'

In [42]:
ids_10_mais_proximos = recomendar(id)

ids_10_mais_proximos

Produto de interesse: e76572c2ae4cede1e3b2ead512423efe
Retornando 10 produtos mais próximos de: e76572c2ae4cede1e3b2ead512423efe.


['ac1fcaf7402bfbf019afdcce1a784ad4',
 'b09cd20111b8c27705b617c3b5af7608',
 '9eacfe990da87a66d7fc95e4f97bad14',
 'e76572c2ae4cede1e3b2ead512423efe',
 'd5bf1e6386a450dd26acf14a6f41768f',
 '001b237c0e9bb435f2e54071129237e9',
 'a755921e6627c919c9693f6b34884767',
 'a73521e768a217ac6a47c1752ceaf71e',
 'a71d0bc0c279b81bda788ecaca585ca5',
 'a6d040e128bd9c3f2f4c04f27f3ffbf6']

Utilizando um ID diferente, ele retorna os top 10 de reviews daquela categoria

In [43]:
id2 = 'id_novo'

In [44]:
ids_10_mais_proximos = recomendar(id2)

ids_10_mais_proximos

Produto de interesse: id_novo
O item id_novo não está incluso na matriz de recomendação. Retornando os 10 mais avaliados


['99a4788cb24856965c36a24e339b6058',
 'f1c7f353075ce59d8a6f3cf58f419c9c',
 '06edb72f1e0c64b14c5b79353f7abea3',
 'ec2d43cc59763ec91694573b31f1c29a',
 '84f456958365164420cfc80fbe4c7fab',
 '777d2e438a1b645f3aec9bd57e92672c',
 '363218ba55c610b750224f90bdd34be1',
 '64fb265487de2238627ce43fe8a67efc',
 '2a2d22ae30e026f1893083c8405ca522',
 'bd6e6fce9ada76ea2db0f1912e8e478f']

O modelo funciona bem e sua aplicação pode ser usada na seguinte situação:
* Quando um usuário entra numa seção da loja e pesquisa um item, aparecendo assim itens da mesma categoria como sugestão para facilitar a pesquisa do usuário.

Porém tem alguns pontos de melhoria que podem ser aplicados
* Os reviews são baseadas no pedido de compra, ou seja, se o usuário fez uma compra de 5 itens, a nota vai para os 5 itens, indiferente se um produto foi mais ou menos satisfatório que o outro
* A média de reviews pode ser uma métrica que engana pois uma venda de um produto com 1 review de 5 pontos tem média maior que 200 vendas de um produto com média de 4.9, porém o último item seria mais popular.
* Algumas categorias da loja possui poucos itens como "cds_dvds_musicais" que possui 1 item, seria interessante para a empresa ter uma distribuição categorica um pouco mais abrangente para evitar possíveis erros na recomendação